[<img src="https://www.scenerepresentations.org/assets/logo/logo+text-negative@2x.png" alt="TODO: make logo work better on Colab background" width="212"/>](https://www.scenerepresentations.org)

[`MIT 6.S980, Machine Learning for Inverse Graphics`](https://www.scenerepresentations.org/courses/inverse-graphics/)
# Homework 3: Prior-based Reconstruction

### Administrative details
This course publishes its assignments as Jupyter notebooks, hosted on Google Colab. This way, you don't need any local development setup. If you're more comfortable with your local dev environment, feel free to download this notebook instead ("File -> Download -> Download .ipynb") and edit it in your local environment. Else, make sure you're logged into your Google Account and click "File -> Save a copy in Drive". You can then edit your copy of the notebook to complete your assignment. You'll find instructions for submitting your work at the [end of this notebook](#scrollTo=Submission_Instructions).


### About this homework

In our last unit, we saw how we can use neural networks as "just another parameterization of continuous functions". In other words, there was nothing here that one would conventionally call "Artificial Intelligence" or even "Machine Learning" - we simply fit the neural network to some function values.

Now, we will move on to something that would more commonly be referred to as "Machine Learning" and "Artificial Intelligence": We will use neural networks to make predictions for inputs *unseen* at training time.

Technically, we also already did that in our previous assignment: Whenever you queried your occupancy network at a coordinate that wasn't in the training set, you "generalized" to an unseen input.

However, since these inputs are low-dimensional, we would be compelled to call this behavior "interpolation", and are not overly surprised that it works, as we can easily build algorithms that will interpolate some nearest neighbors in the training set and get a similar performance.

The key difference now will be that the inputs will be high-dimensional (think >10,000 dimensions). In such high-dimensional spaces, it is *not* straight-forward to hand-craft an algorithm that will assign a reasonable value to an input unobserved at training time.

This, then, is what deep learning is, to date, *uniquely* good at, and is the property that has enabled them to revolutionize computer vision and NLP.

### Contributors
- _Prof. Vincent Sitzmann, Scene Representation Group._
- _Prafull Sharma_

# Setup

In [ ]:
!pip install einops

import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import IterableDataset
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, List, Optional, Tuple, Generator, Dict
import os
import h5py
import random

import imageio
import os
import skimage
import h5py
import io

import math
from einops import rearrange, repeat

print(f"Installed Torch version: {torch.__version__}")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print(device)


def init_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d) or isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)


### Testing your code

For your convenience, we have written a little test function, "check_function", which will serve as a unit-test for _every function that you will be graded on_. This will allow you a simple, shallow check whether your function does something reasonable.

Note that it is **not guaranteed** that you will get full score if all of your functions pass this test - in our grading script, we test your functions much more thoroughly. It is your responsibility to make sure your implementation is correct.

In [ ]:
def check_function(test_name, function_name, test_input, test_output):
    try:
        student_output = function_name(*test_input)
    except TypeError as error:
        print("Function", test_name, "has a error and didn't run cleanly. Error:", error)
        return False
    if isinstance(student_output, tuple):
        student_output = list(student_output)
    else:
        student_output = [student_output]

    for i in range(len(test_output)):
        if not torch.allclose(student_output[i], test_output[i], rtol=1e-03):
            print(test_name, ": Your function DOES NOT work.")
            return False
    print(test_name, ": Your function works!")
    return True

def check_losses(test_name, losses_array, loss_threshold):
    average_loss_last5 = np.mean(losses[-5:])
    if average_loss_last5 >= loss_threshold:
        print(test_name, ": Your function isn't optimized well")
        return False
    else:
        print(test_name, ": Your function works!")
    return True

def check_shape(test_name, your_shape, expected_shape):
    if your_shape != expected_shape:
        print(test_name, ": Your function DOES NOT output a tensor with expected shape")
        return False
    else:
        print(test_name, ": Your function outputs the expected shape!")
    return True

# Part 1: Convolutional Auto-Decoders

As a training exercise, we will build a convolutional auto-decoder to reconstruct images of MNIST digits.

The techniques we will learn will apply 1:1 to Part 2, where we will build auto-decoders for 3D scenes!

The MNIST dataset is conveniently available in Pytorch:

In [ ]:
mnist = datasets.MNIST('mnist_data', train=True, download=True,
                        transform=transforms.Compose([
                            torchvision.transforms.Pad(18, fill=0, padding_mode='constant'),
                            transforms.ToTensor()
                        ]))
# The dataset yields tuples of (image, digit_class)
img, digit = next(iter(mnist))

print(digit)
plt.imshow(img.squeeze())
plt.show()

## Part 1.1: Writing a Convolutional Decoder

First, we will write a convolutional decoder: This is a module that will take in a feature map of size `(batch, in_ch, height, width)` and up-sample it `num_up` times to yield a feature map of size `(batch, out_ch, height*(2^num_up), width*(2^num_up)`.

**DISCLAIMER: For upsampling, we will use transpose convolutions. Note that this is, generally, a bad idea - usually, you would use a conditional convolutional network like in the StyleGan v2 decoder, but that is not the focus of this class, and we didn't talk about it, so we're going with this :) Note that that wouldn't change the input-output behavior of this module, it would just replace the transpose convolutions with FiLM / Ada-In conditioned convolutions, but we digress.**

Implement the convolutional decoder with transpose convolutions below.

In [ ]:
class CoordCat(nn.Module):
    """
    This class takes an input of shape (B, L, ...) and concatenates normalied
    coordinates along the latent dimension. This is followed when we need
    positional information, such as in the case of CoordConv. We will use this
    when we write the decoder in ConvDecoder.
    """
    def __init__(self, mode="2d"):
        super().__init__()
        self.mode = mode

    def forward(self, input):
        if self.mode == "2d":
            x = torch.linspace(-1, 1, input.shape[2])
            y = torch.linspace(-1, 1, input.shape[3])
            xy = torch.meshgrid(x, y)
            xy = torch.stack(xy, dim=0)
            out = xy.repeat(input.shape[0], 1, 1, 1)
        else:
            x = torch.linspace(-1, 1, input.shape[2])
            y = torch.linspace(-1, 1, input.shape[3])
            z = torch.linspace(-1, 1, input.shape[4])
            xyz = torch.meshgrid(x, y, z)
            xyz = torch.stack(xyz, dim=0)
            out = xyz.repeat(input.shape[0], 1, 1, 1, 1)
        return torch.cat([input, out.to(input.device)], dim=1)

class ConvDecoder(nn.Module):
    def __init__(self, in_ch, hidden_ch, num_up, out_ch, mode='2d'):
        super().__init__()

        """
        In this function, we will construct a convolutional decoder based on the
        input mode and other input parameters specifying the input, hidden, and
        output channels. num_up specifies the number of layers in the decoder.

        For mode is '2d', we use nn.Conv2d to perform the initial and final
        convolutions, and nn.Conv3d is used when mode is '3d'.

        We use transposed convolutions to upscale the input. We will use
        nn.ConvTranspose2d when mode is '2d' and nn.ConvTranspose3d when mode is
        '3d'. This is used to decode the input into a 2D or 3D feature tensor.

        Note: Do not call '.to(device)' on any modules defined here.
              All modules are automatically mounted to device when initiated
              and mounted on device.
        """

        if mode=='2d':
            conv = nn.Conv2d
            transpose_conv = nn.ConvTranspose2d
            input_hidden_channels = hidden_ch + 2
        elif mode == '3d':
            conv = nn.Conv3d
            transpose_conv = nn.ConvTranspose3d
            input_hidden_channels = hidden_ch + 3

        ###
        # TODO
        # Specify in_conv as a nn.Sequential with (conv, LeakyReLU) where conv
        # is the convolution operation defined above. This uses the input
        # channels as in_ch and outputs hidden_ch channels with stride of 1 and
        # padding specified as 'same' with 'zeros'. Make sure to perform
        # LeakyReLU inplace and use negative_slope of 0.2.

        self.in_conv = #### YOUR CODE HERE ####

        # Hidden transposed convolution layers
        # One way to define nn.Sequential module is nn.Sequential(*arr) where
        # arr is a list containing nn.Module units. Here, we will append
        # (CoordCat, transpose_conv, LeakyReLU), 'num_up' times. CoordCat needs
        # to mode as the argument. Each of the transpose_conv use
        # input_hidden_channels as the number of input channels and input and
        # output hidden_ch channels using a kernel size of 3, stride
        # of 2, padding as 1, and output_padding as 1.
        hidden_conv = []

        ######################## YOUR CODE HERE ################################

        self.hidden_conv = nn.Sequential(*hidden_conv)

        # Specify out_conv using conv with hidden_ch input channels and outputting
        # out_ch with kernel size as 3, padding as 'same', and padding_mode as
        # 'zeros'.

        self.out_conv = #### YOUR CODE HERE ####

        # TODO
        ###

        self.apply(init_weights)

    def forward(self, input):
        # TODO
        # Call in_conv, hidden_conv, and out_conv modules on the input.

        #### YOUR CODE HERE ####


def test_conv_decoder():
    test_in = torch.randn((1, 4, 32, 32))
    mod = ConvDecoder(in_ch = 4, hidden_ch = 32, num_up = 3, out_ch = 4)
    return mod(test_in)

check_shape("ConvDecoder",
            test_conv_decoder().shape,
            torch.Size([1, 4, 256, 256]))

## Part 1.2: Decoding a single latent code to an image.

The convolutional decoder enables us to take a single latent code of shape (1, ch, 1, 1) and learn an MLP to upsample it to a full image.

Below, write a class that accomplishes this `LatentFeatureGrid`:

In [ ]:
import torch.nn.functional as F

class LatentFeatureGrid(nn.Module):
    def __init__(self, latent_sidelength, latent_ch, num_up, out_ch, mode='2d'):
        super().__init__()

        if mode == '2d':
            latent_shape = 1, latent_ch, latent_sidelength, latent_sidelength
        elif mode == '3d':
            latent_shape = 1, latent_ch, latent_sidelength, latent_sidelength, latent_sidelength

        #######
        # TODO
        # Generate an nn.Parameter with shape (1, out_dim, *resolution_per_dim)
        # called self.latent_grid with small positive random numbers.

        self.latent_grid = #### YOUR CODE HERE ####

        # Instatiate decoder as ConvDecoder which takes in latent_ch channels as
        # input and output, and the num_up and mode specified in the arguments.

        self.decoder = #### YOUR CODE HERE ####

        # TODO
        ######

    def forward(self, input=None):
        '''
        coordinate: (batch_size, num_points, 2)
        '''
        # TODO
        # return the output of the decoder on input of self.latent_grid.
        #### YOUR CODE HERE ####

## Part 1.3: Fitting a single latent to a single image.

In homework 1, we fit an image grid to an image grid. Now, we will do something similar: we will fit a low-dimensional feature grid to an image.

Let's retrieve a single image from our mnist dataset, and wrap it in a little generator that will always yield exactly that one image:

In [ ]:
image, _ = next(iter(mnist))

def image_generator():
    while True:
        # We have to yield "none" as the first entry of the tuple, as our
        # Fitting function expects some kind of model_input
        yield None, torch.tensor(image, device=device)[None]

We now create the function `fit`, which defines the training loop.



In [ ]:
import matplotlib.pyplot as plt
import collections
%matplotlib inline
%config InlineBackend.figure_format='retina'

def to_gpu(ob):
    if isinstance(ob, collections.abc.Mapping):
        return {k: to_gpu(v) for k, v in ob.items()}
    elif isinstance(ob, tuple):
        return tuple(to_gpu(k) for k in ob)
    elif isinstance(ob, list):
        return [to_gpu(k) for k in ob]
    else:
        try:
            return ob.cuda()
        except:
            return ob

def fit(
    model: nn.Module,
    data_iterator,
    loss_fn,
    resolution: Tuple,
    optimizer,
    plotting_function = None,
    steps_til_summary = 500,
    total_steps=2001
   ):

    losses = []
    for step in range(total_steps):
        # Get the next batch of data and move it to the GPU
        model_input, ground_truth = next(data_iterator)
        model_input, ground_truth = to_gpu(model_input), to_gpu(ground_truth)

        # Compute the MLP output for the given input data and compute the loss
        model_output = model(model_input)

        # Implement a simple mean-squared-error loss between the
        loss = loss_fn(model_output, ground_truth, model) # Note: loss now takes "model" as input.

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the losses so that we can plot them later
        # losses.append(loss.detach().cpu().numpy())

        # Every so often, we want to show what our model has learned.
        # It would be boring otherwise!
        if not step % steps_til_summary:
            print(f"Step {step}: loss = {float(loss.detach().cpu()):.5f}")

            if plotting_function is not None: # Note: we now call the "plotting function" instead of hard-coding the plotting here.
                plotting_function(model_output, ground_truth, resolution)

    return model_output

As loss, we will use a simple mse loss for now. Note that it now must accept the "model" variable as input.

In [ ]:
def mse_loss(
    mlp_out,
    gt,
    model: nn.Module # We won't be using this one here, but later, we will!
    ):
    return ((mlp_out - gt)**2).mean()

Let's implement the plotting function for this single-image fitting experiment. It will simply take the model output (which in this case will be the image) and plot it next to the ground truth.

In [ ]:
def plot_output_ground_truth(model_output, ground_truth, resolution):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), squeeze=False)
    axes[0, 0].imshow(model_output.cpu().view(*resolution).detach().numpy())
    axes[0, 0].set_title("Trained MLP")
    axes[0, 1].imshow(ground_truth.cpu().view(*resolution).detach().numpy())
    axes[0, 1].set_title("Ground Truth")

    for i in range(2):
        axes[0, i].set_axis_off()

    plt.show()

We instantiate our model, as well as an optimizer:

In [ ]:
# TODO
# Define latent_img as LatentFeatureGrid with latent_sidelength as 2, latent_ch
# as 128, outputting 1 channel, using 5 upsampling layers (num_up). Mount
# the latent_img on the gpu.

latent_img = #### YOUR CODE HERE ####

# Define optim as Adam optimizer that optimizes the parameters of latent_img
# with a learning rate of 1e-4

optim = #### YOUR CODE HERE ####

And can now call the `fit` function!

In [ ]:
# TODO
# Call the fit function using latent_img as the model, image_generator as the
# data_iteratorm, mse_loss as the loss_fn. We will optimize with a image resolution
# of (64, 64) for 501 steps as fitting an image is relatively quick!
# Don't forget to pass the optimize and plot_output_groun_truth for plotting.

final_out = #### YOUR CODE HERE ####

Ok - so far, nothing new. The fact that this works should not surprise you!

Now, let's make this more interesting. Let's learn a *space* of MNIST images!

In [ ]:
del latent_img

## Part 1.4: Building a convolutional auto-decoder for images

The `LatentImgGrid` module above only stores a *single* latent grid that can only represent a *single* image. This means that we cannot reconstruct an image from incomplete observations - for instance, observing only half the digit.

Now, we will learn a *latent space* of digits, such that we can reconstruct a digit from a *partial* observations.

We will do that using the *auto-decoder* framework discussed in the lecture. Given a dataset of images $\{ I_i \}_{i=1}^N$, we will randomly initialize a corresponding set of latent grids $\{ \mathbf{z}_i \}_{i=1}^N$.

We will then solve the following optimization problem:

$$
    min_{\{\mathbf{z}_i\}} \sum_i \| D(\mathbf{z}_i) - I_i \|^2
$$

where $D(\cdot)$ is our convolutional decoder.

### Part 1.4.1 Writing an image auto-decoder

To build an auto-decoder, we will modify our `LatentFeatureGrid` class from above to have a *set* of latent codes instead of just a single latent code.

In a forward pass, our model will now take as input a *scene idx* idx, retrieve the corresponding latent code, and decode that latent code only.

Fill in the blanks below:

In [ ]:
class ImageAutoDecoder(nn.Module):
    def __init__(self, latent_sidelength, latent_ch, num_up, out_ch, num_latents):
        super().__init__()
        #######
        # TODO

        # Generate an nn.Embedding with num_latents each of dimension as
        # latent_ch * latent_sidelength * latent_sidelength

        self.latents = #### YOUR CODE HERE ####

        # Initialize the embedding using a normal distribution.
        # We can directly do this by calling self.latents.weight.data.normal_
        # with the mean of 0 and standard deviation as 1e-1.

        ########################## YOUR CODE HERE #############################


        # Define decoder as a ConvDecoder which takes in latent_ch channel input
        # and outputs a out_ch channel output. We will use num_up upsampling
        # layers, each with hidden dimension of 128.

        self.decoder = #### YOUR CODE HERE ####

        # TODO
        ######

        self.latent_ch = latent_ch
        self.latent_sl = latent_sidelength

    def forward(self, idcs):
        '''
        idcs: (batch_size, 1)
        '''
        # TODO
        # Using the input indices 'idcs', get the respective latents
        latents = #### YOUR CODE HERE ####

        # Reshape the latents to (batch_size, latent_ch, latent_sidelength, latent_sidelength)
        latents = #### YOUR CODE HERE ####

        # Compute the output of decoder of the latents
        outputs = #### YOUR CODE HERE ####
        return outputs, latents
        # TODO

We can now call this module with an input that is a tensor of integer values as follows:

In [ ]:
def test_autodecoder_class():
    img_ad = ImageAutoDecoder(4, 32, num_up=4, out_ch=1, num_latents=100).cuda()

    idx = torch.tensor([0], device=device).int()
    img_ad(idx)

test_autodecoder_class()

### Part 1.4.2 Training a space of MNIST digits

Now, let's train a latent space of MNIST digits! First, we'll need to prep a dataset. The dataset will need to yield as model input the index of the respective image, and as ground truth the image.

Fill in the blanks:

In [ ]:
class MNISTAutoDecData(torch.utils.data.IterableDataset):
    def __init__(self, num_images):
        self.mnist = datasets.MNIST('mnist_data', train=True, download=True,
                                transform=transforms.Compose([
                                    torchvision.transforms.Pad(18, fill=0, padding_mode='constant'),
                                    transforms.ToTensor()
                                ]))
        self.num_images = num_images

    def __iter__(self):
        while True:
            #######
            # TODO
            # Get a random integer in range [0, self.num_images)
            rand_idx = #### YOUR CODE HERE ####

            # get the img and digit from mnist dataset
            img, digit = #### YOUR CODE HERE ####

            # TODO
            ######

            # yield tensor with rand_idx and a tuple of (image, digit)
            yield torch.tensor([rand_idx]), (img, digit)


We will now decide on the number of images we would like to auto-decode. Let's go with 10,000:

In [ ]:
num_imgs = 10_000
dataset = MNISTAutoDecData(num_imgs)
loader = torch.utils.data.DataLoader(dataset, batch_size=64)

Let's initialize our auto-decoding model with an according number of latents:

In [ ]:
# TODO
# Create an ImageAutoDecoder with sidelength 1, latent_ch as 64, out_ch as 1,
# and number of latents as num_imgs (we want one for each image). Mount the model
# on the gpu using .cuda().
img_ad = #### YOUR CODE HERE ####

Let's test a single batch:

In [ ]:
idx, (img, digit) = next(iter(loader))
test_out = img_ad(idx.cuda())

del test_out

Now, we'll have to write a loss for the auto-decoder. This loss is a bit more involved. It has three parts:
1. An image loss - that's just the reconstruction loss.
2. An l2 loss on the latents. This encourages the latent space to be smooth by forcing all the latents to stay close to the origin.
3. A small l2 loss on the parameters of the convolutional decoder. This prevents overfitting of the decoder.

**This is why we need to pass the model as a parameter to the loss function - so that we can enforce a small l2 regularizer on the model parameters.**

Fill in the blanks below:

In [ ]:
def autodecoder_loss(mlp_out, gt, model):

    img, latents = mlp_out

    # TODO
    # Image loss: Compute the mean squared error loss using the img and gt[0]
    img_loss = #### YOUR CODE HERE ####

    # Latent loss: We want the latents to be small valued which can be achieved
    # by adding the mean-square of the values of the latent. This is also called
    # sparisity loss as it promotes the values of the latents to be close to 0.
    latent_loss = #### YOUR CODE HERE ####

    # L2 weight regularization: Similarly, we want the weights of the model to
    # sparse as well. Loop over all the parameters of the model and add the
    # mean-square of the parameters to param_loss.
    # Look at model.named_parameters() and make sure not to apply the loss if
    # 'latent' is in the name of the parameter.
    param_loss = 0.

    #### YOUR CODE HERE ####

    return param_loss*1e-2 + latent_loss*1e-2 + img_loss

We also need to write a new function plotting function, as the `model_output` is now a tuple.

All we have to do is to only plot the first part of the tuple:

In [ ]:
def plot_output_ground_truth(model_output, ground_truth, resolution):
    img, _ = model_output

    fig, axes = plt.subplots(1, 2, figsize=(12, 6), squeeze=False)
    axes[0, 0].imshow(img[0].cpu().view(*resolution).detach().numpy())
    axes[0, 0].set_title("Trained MLP")
    axes[0, 1].imshow(ground_truth[0][0].cpu().view(*resolution).detach().numpy())
    axes[0, 1].set_title("Ground Truth")

    for i in range(2):
        axes[0, i].set_axis_off()

    plt.show()


Now, let's fit!

**Note that this training takes quite a bit longer than prior experiments (~10 minutes) - that's because this time, our neural network actually has to learn how to compress information!**

As you're training, pay attention to how the images look vs. ground truth. One thing you should see is that *the auto-decoder gets the digit right long before it gets the exact shape right*. What I mean by that is that the output might look like a 4, but it looks like a *different* 4 than the ground-truth one. That's a good sign that our latent space is working, and that all the fours lie close to each other in latent space!

In [ ]:
# TODO
# Define optim as Adam optimizer which optimizes the parameters of img_ad with
# a learning rate of 1e-3.
optim = #### YOUR CODE HERE ####

_ = fit(img_ad,
        iter(loader),
        loss_fn=autodecoder_loss,
        resolution=(64, 64),
        plotting_function=plot_output_ground_truth,
        optimizer=optim,
        total_steps=10001
        )

Hint: the loss should converge to approximately ~ 1e-3 after 10000 steps, and the digits should be clearly legible.

### Part 1.4.3: Reconstructing a partial MNIST digit

What have we gained? Two things: we have learned an expressive latent space, and we have gained the ability to reconstruct digits from incomplete observations.

Specifically:
1. We are given the *left half* of an image.
2. We will initialize a new latent code, randomly.
3. Freezing the weights of our model, we will only backpropagate into the latent code to make the left half of the images match.

First, let's build a loss that only penalizes the left half of the image:

In [ ]:
def left_half_loss(mlp_out, gt, model):
    img, _ = mlp_out

    # TODO
    # Compute the pixel-wise squared difference between img and gt[0]
    diff = #### YOUR CODE HERE ####

    # print("diff shape:", diff.shape)
    # Define the half as the left half of the diff
    half = #### YOUR CODE HERE ####

    return half.mean()
    # TODO

Next, let's re-initialize the latent grid of our model as follows:

In [ ]:
with torch.no_grad():
    img_ad.latents.weight.data.zero_()

Now, we only need to build a optimizer that *only optimizes the latent grid, leaving the convolutional decoder untouched*.

**This is why we modified the `fit` function to take the optimizer as input - we now don't want to optimize all the parameters anymore, just a subset!**

In [ ]:
def return_optimizer():
    # TODO
    # return an optimizer that optimizes img_ad.latents.weights with a learning
    # rate of 1e-4
    return #### YOUR CODE HERE ####

optim = return_optimizer()

Let's randomly sample an mnist image and fit!

In [ ]:
def test_dataset():
    idx, (img, digit) = next(iter(MNISTAutoDecData(num_imgs)))
    while True:
        yield idx, (img, digit)

dataset = test_dataset()

In [ ]:
# TODO
# Call the fit function to fit img_ad using the left_half_loss function using
# optim as the optimizer. Train this for 10001 steps.

#### YOUR CODE HERE ####

img, latent = img_ad(torch.tensor([100]).cuda())

It works! Note how it completes the rest of the digit, even though it is only observing the left half.

Note that the regularization and initialization of the latent codes in this case are critical. For instance, try training the auto-decoder when the latent codes are initialized with a large standard deviation: What you'll find is that you can still fit the training set just fine, but in-painting will *not* work! That's because it's critical that we force the latent space to be compact, i.e., that all the latent codes are close together and tightly packed. L2 regularization of weights and latent codes plays a similar role. If you run into troubles in your project, these are good things to investigate in a toy setup!

In [ ]:
del img_ad
del mnist
del optim

# Part 2: Training an auto-decoder of voxelgrid scene representations

Next, we will do the same thing just in 3D and with neural rendering!

## Part 2.0: Setup

First, we'll need a dataset of images of 3D scenes and their camera poses. For that, we'll use the dataset from the paper "Scene Representation Networks, Sitzmann et al. 2019":

In [1]:
import os

if not os.path.exists("/content/cars_train.hdf5"):
    # Download SRNs-cars dataset
    !gdown 1SBjlsizq0sFNkCZxMQh-pNRi0HyFozKb

Downloading...
From: https://drive.google.com/uc?id=1SBjlsizq0sFNkCZxMQh-pNRi0HyFozKb
To: /content/cars_train.hdf5
100% 799M/799M [00:05<00:00, 151MB/s] 


Next, below, you can find all the multi-view functions again that you already know from Assignments 1 & 2:

In [2]:
def homogenize_points(points: torch.Tensor):
    """Appends a "1" to the coordinates of a (batch of) points of dimension DIM.

    Args:
        points: points of shape (..., DIM)

    Returns:
        points_hom: points with appended "1" dimension.
    """
    ones = torch.ones_like(points[..., :1], device=points.device)
    return torch.cat((points, ones), dim=-1)


def homogenize_vecs(vectors: torch.Tensor):
    """Appends a "0" to the coordinates of a (batch of) vectors of dimension DIM.

    Args:
        vectors: vectors of shape (..., DIM)

    Returns:
        vectors_hom: points with appended "0" dimension.
    """
    zeros = torch.zeros_like(vectors[..., :1], device=vectors.device)
    return torch.cat((vectors, zeros), dim=-1)


def unproject(
    xy_pix: torch.Tensor,
    z: torch.Tensor,
    intrinsics: torch.Tensor
    ) -> torch.Tensor:
    """Unproject (lift) 2D pixel coordinates x_pix and per-pixel z coordinate
    to 3D points in camera coordinates.

    Args:
        xy_pix: 2D pixel coordinates of shape (..., 2)
        z: per-pixel depth, defined as z coordinate of shape (..., 1)
        intrinscis: camera intrinscics of shape (..., 3, 3)

    Returns:
        xyz_cam: points in 3D camera coordinates.
    """
    xy_pix_hom = homogenize_points(xy_pix)
    xyz_cam = torch.einsum('...ij,...kj->...ki', intrinsics.inverse(), xy_pix_hom)
    xyz_cam *= z
    return xyz_cam


def transform_world2cam(xyz_world_hom: torch.Tensor, cam2world: torch.Tensor) -> torch.Tensor:
    """Transforms points from 3D world coordinates to 3D camera coordinates.

    Args:
        xyz_world_hom: homogenized 3D points of shape (..., 4)
        cam2world: camera pose of shape (..., 4, 4)

    Returns:
        xyz_cam: points in camera coordinates.
    """
    world2cam = torch.inverse(cam2world)
    return transform_rigid(xyz_world_hom, world2cam)


def transform_cam2world(xyz_cam_hom: torch.Tensor, cam2world: torch.Tensor) -> torch.Tensor:
    """Transforms points from 3D world coordinates to 3D camera coordinates.

    Args:
        xyz_cam_hom: homogenized 3D points of shape (..., 4)
        cam2world: camera pose of shape (..., 4, 4)

    Returns:
        xyz_world: points in camera coordinates.
    """
    return transform_rigid(xyz_cam_hom, cam2world)


def transform_rigid(xyz_hom: torch.Tensor, T: torch.Tensor) -> torch.Tensor:
    """Apply a rigid-body transform to a (batch of) points / vectors.

    Args:
        xyz_hom: homogenized 3D points of shape (..., 4)
        T: rigid-body transform matrix of shape (..., 4, 4)

    Returns:
        xyz_trans: transformed points.
    """
    return torch.einsum('...ij,...kj->...ki', T, xyz_hom)


def get_unnormalized_cam_ray_directions(xy_pix:torch.Tensor,
                                        intrinsics:torch.Tensor) -> torch.Tensor:
    return unproject(xy_pix, torch.ones_like(xy_pix[..., :1], device=xy_pix.device),  intrinsics=intrinsics)


def get_world_rays(xy_pix: torch.Tensor,
                   intrinsics: torch.Tensor,
                   cam2world: torch.Tensor,
                   ) -> torch.Tensor:
    # Get camera origin of camera 1
    cam_origin_world = cam2world[..., :3, -1]

    # Get ray directions in cam coordinates
    ray_dirs_cam = get_unnormalized_cam_ray_directions(xy_pix, intrinsics)

    # Homogenize ray directions
    rd_cam_hom = homogenize_vecs(ray_dirs_cam)

    # Transform ray directions to world coordinates
    rd_world_hom = transform_cam2world(rd_cam_hom, cam2world)

    # Tile the ray origins to have the same shape as the ray directions.
    # Currently, ray origins have shape (batch, 3), while ray directions have shape
    cam_origin_world = repeat(cam_origin_world, 'b ch -> b num_rays ch', num_rays=ray_dirs_cam.shape[1])

    # Return tuple of cam_origins, ray_world_directions
    return cam_origin_world, rd_world_hom[..., :3]


def get_opencv_pixel_coordinates(
    y_resolution: int,
    x_resolution: int,
    ):
    """For an image with y_resolution and x_resolution, return a tensor of pixel coordinates
    normalized to lie in [0, 1], with the origin (0, 0) in the top left corner,
    the x-axis pointing right, the y-axis pointing down, and the bottom right corner
    being at (1, 1).

    Returns:
        xy_pix: a meshgrid of values from [0, 1] of shape
                (y_resolution, x_resolution, 2)
    """
    i, j = torch.meshgrid(torch.linspace(0, 1, steps=x_resolution),
                          torch.linspace(0, 1, steps=y_resolution))

    xy_pix = torch.stack([i.float(), j.float()], dim=-1).permute(1, 0, 2)
    return xy_pix

NameError: ignored

## Part 2.1: Dataset class for SRN cars

We'll have to write a dataset loader for the SRN dataset.

What will this dataset have to yield? Just like in the MNIST example above, it will have to yield *images* and *indices of the corresponding car*, so that the auto-decoder knows which latent it should use. Note that now, there are *several images* for *one latent code*, b/c the latent codes are now matched up with the 3D scenes, not with the images!

Further, just like in Assignment 2, our dataset will have to yield *camera extrinsic and intrinsic parameters*, as well as *pixel coordinates*.

We bundle up all of these things in a single dictionary of the form

```python
model_input = {'cam2world': c2ws, # (4,4) cam2world camera pose
                'intrinsics': intrinsics, # (3,3) camera intrinsics
                'x_pix': x_pix, # (H*W, 2) opencv pixel coordinates
                'idx': torch.tensor([idx]) # (1) index of the 3D scene, torch integer tensor
                }
```



We wrote most of it for you already, but there's a few blanks we'd like you to fill in in the `__iter__` function:

In [ ]:
from skimage.transform import resize

def parse_rgb(hdf5_dataset):
    s = hdf5_dataset[...].tobytes()
    f = io.BytesIO(s)

    img = imageio.imread(f)[:, :, :3]
    img = skimage.img_as_float32(img)
    return img


def parse_intrinsics(hdf5_dataset):
    s = hdf5_dataset[...].tobytes()
    s = s.decode('utf-8')

    lines = s.split('\n')
    f, cx, cy, _ = map(float, lines[0].split())
    full_intrinsic = torch.tensor([[f, 0., cx],
                                    [0., f, cy],
                                    [0., 0, 1]])

    return full_intrinsic


def parse_pose(hdf5_dataset):
    raw = hdf5_dataset[...]
    ba = bytearray(raw)
    s = ba.decode('ascii')

    lines = s.splitlines()
    pose = np.zeros((4, 4), dtype=np.float32)

    for i in range(16):
        pose[i // 4, i % 4] = lines[0].split(" ")[i]

    pose = torch.from_numpy(pose.squeeze())
    return pose


class SRNsCars(IterableDataset):
    def __init__(self, max_num_instances=None, img_sidelength=None):
        self.f = h5py.File('cars_train.hdf5', 'r')
        self.instances = sorted(list(self.f.keys()))

        self.img_sidelength = img_sidelength

        if max_num_instances:
            self.instances = self.instances[:max_num_instances]

    def __len__(self):
        return len(self.instances)

    def __iter__(self, override_idx=None):
        while True:
            if override_idx is not None:
                idx = override_idx
            else:
                idx = random.randint(0, len(self.instances)-1)

            key = self.instances[idx]

            instance = self.f[key]
            rgbs_ds = instance['rgb']
            c2ws_ds = instance['pose']

            rgb_keys = list(rgbs_ds.keys())
            c2w_keys = list(c2ws_ds.keys())

            observation_idx = np.random.randint(0, len(rgb_keys))
            rgb = parse_rgb( rgbs_ds[rgb_keys[observation_idx]] )

            x_pix = get_opencv_pixel_coordinates(*rgb.shape[:2])

            # There is a lot of white-space around the cars - we'll thus crop the images a bit:
            rgb = rgb[32:-32, 32:-32]
            x_pix = x_pix[32:-32, 32:-32]

            # Nearest-neighbor downsampling of *both* the
            # RGB image and the pixel coordinates. This is better than down-
            # sampling RGB only and then generating corresponding pixel coordinates,
            # which generates "fake rays", i.e., rays that the camera
            # didn't actually capture with wrong colors. Instead, this simply picks a
            # subset of the "true" camera rays.
            if self.img_sidelength is not None and rgb.shape[0] != self.img_sidelength:
                rgb = resize(rgb,
                             (self.img_sidelength, self.img_sidelength),
                             anti_aliasing=False,
                             order=0)
                rgb = torch.from_numpy(rgb)
                x_pix = resize(x_pix,
                               (self.img_sidelength, self.img_sidelength),
                               anti_aliasing=False,
                               order=0)

            x_pix = rearrange(x_pix, 'i j c -> (i j) c')
            c2w = parse_pose( c2ws_ds[c2w_keys[observation_idx]] )

            rgb = rearrange(rgb, 'i j c -> (i j) c')

            intrinsics = parse_intrinsics( instance['intrinsics.txt'] )
            intrinsics[:2, :3] /= 128. # Normalize intrinsics from resolution-specific intrinsics for 128x128

            ###
            # TODO
            # Create a dictionary which contains the following
            # 1. the 'cam2world' poses, which we computed as c2w
            # 2. the camera 'intrinsics'
            # 3. the pixel coordinates, 'x_pix'
            # 4. the index of the sampled car, 'idx'
            model_input = #### YOUR CODE HERE ####

            yield model_input, rgb
            # TODO
            ###

Let's take a look at a single sample:

In [ ]:
sl = 32
dataset = SRNsCars(img_sidelength=sl)
mi, rgb = next(iter(dataset))

rgb = rgb.view(sl, sl, 3)

plt.imshow(rgb)
plt.show()

print("Cam2world ", mi['cam2world'])
print("Intrinsics ", mi['intrinsics'])
print("Pixel coords shape", mi['x_pix'].shape, mi['x_pix'].max())
print("Scene idx", mi['idx'].shape)

***Please make sure that this all makes sense to you.***

## Part 2.2: Latent 3D Grid, Decoder, and Radiance Field

We will now write the 3D equivalent of the LatentImgGrid, as well as a Radiance Field that will use that latent voxel grid for rendering.

In [ ]:
class LatentHybridVoxelNeuralField(nn.Module):
    def __init__(self, feature_dim, out_dim, mode='bilinear'):
        super().__init__()
        self.mode = mode

        ######
        # TODO
        # Initialize the grid as LatentFeatureGrid with side_length of 1,
        # latent dimension as the feature_dim, num_up as 5, and out_ch as out_dim.
        # Note that in this case, we will use the mode as '3d' for the grid.

        self.grid = #### YOUR CODE HERE ####

        # Use nn.Sequential to generate an mlp with 2 layers:
        # (1) (linear, leaky_relu) with feature_dim inputs and feature_dim outputs
        # (2) (linear) with feature_dim inputs and out_dim outputs

        self.mlp = #### YOUR CODE HERE ####

        # TODO
        ######

        self.apply(init_weights)

    def forward(self, coordinate):
        ######
        # TODO

        # Call self.grid to decode latent code into 3D grid.
        # Recall that you don't need to pass any input as the decoder is
        # evaluated on the latent which is a parameter in the LatentFeatureGrid.
        grid = #### YOUR CODE HERE ####

        # print("Coordinate: ", coordinate.min(), coordinate.max())


        # Sample 3D grid at coordinate with grid_sample.
        # Remember to add dimensions to make it compatible with grid_sample
        coord = #### YOUR CODE HERE ####

        # TODO
        ######

        # Grid_sample does not broadcast, so we have to tile the latent grid
        # in case we're fitting a single object but have a batch size larger than 1.
        if grid.shape[0] != coord.shape[0]:
            grid = grid.repeat(coord.shape[0], 1, 1, 1, 1)

        ###
        # TODO
        # Use grid_sample to get the values for the coord in the grid.
        # Ensure that you align_corners when using grid_sample.
        values = #### YOUR CODE HERE ####

        # Use squeeze and permute to ensure the shape of values is (B, -1, L)
        # where B is the batch size and L is the feature_dim
        values = #### YOUR CODE HERE ####

        # compute output by evaluating the mlp on the values.
        output = #### YOUR CODE HERE ####
        return output
        # TODO
        ######


class LatentRadField(nn.Module):
    def __init__(self, n_features=128):
        super().__init__()

        ######
        # TODO
        # scene_rep is a LatentHybridVoxelNeuralField with n_features as input
        # and output feature channels.
        self.scene_rep = LatentHybridVoxelNeuralField(n_features, n_features)

        # Write a (LeakyReLU, linear, ReLU) MLP "self.sigma" that will take as input
        # the output of the voxel neural field and output a scalar density. Use
        # negative_slope of 0.2 for LeakyReLU.
        self.sigma = #### YOUR CODE HERE ####

        # Write a (LeakyReLU, linear, Sigmoid) MLP "self.color" that will take as input
        # the output of the voxel neural field and output a 3 channel density.
        # Use negative_slope of 0.2 for LeakyReLU
        self.radiance = #### YOUR CODE HERE ####

        # TODO
        ######

        self.apply(init_weights)

    def forward(
        self,
        xyz: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
      '''
      Queries the representation for the density and color values
      '''
      ###
      # TODO
      # Do a forward pass through the scene representation.
      features = #### YOUR CODE HERE ####

      # Do a forward pass through both the self.sigma and self.color MLPs
      # to yield sigma and color.
      sigma = #### YOUR CODE HERE ####
      rad = #### YOUR CODE HERE ####

      # TODO
      ###

      # Return sigma and color.
      return sigma, rad

## Part 2.3: Sanity Check: Overfitting a single SRN car

Before we will move on to generalization and prior-based reconstruction, it is always a good idea to reconstruct a single example of your training set to make sure camera poses etc. are all correct.

We will further use this code as a starting point to build our generalizable architecture.

Below, find a more compact implementation of the differentiable renderer, the voxelgrid scene representation, and the radiance field from assignment 2:

In [ ]:
def sample_points_along_rays(
    near_depth: float,
    far_depth: float,
    num_samples: int,
    ray_origins: torch.Tensor,
    ray_directions: torch.Tensor,
    device: torch.device
):
    # Compute a linspace of num_samples depth values beetween near_depth and far_depth.
    z_vals = torch.linspace(near_depth, far_depth, num_samples, device=device)

    # Using the ray_origins, ray_directions, generate 3D points along
    # the camera rays according to the z_vals.
    pts = ray_origins[...,None,:] + ray_directions[...,None,:] * z_vals[...,:,None]

    return pts, z_vals


def volume_integral(
    z_vals: torch.tensor,
    sigmas: torch.tensor,
    radiances: torch.tensor
) -> Tuple[torch.tensor, torch.tensor]:

    # Compute the deltas in depth between the points.
    dists = torch.cat([
        z_vals[..., 1:] - z_vals[..., :-1],
        torch.broadcast_to(torch.Tensor([1e10]).to(z_vals.device), z_vals[...,:1].shape)
        ], -1)

    # Compute the alpha values from the densities and the dists.
    # Tip: use torch.einsum for a convenient way of multiplying the correct
    # dimensions of the sigmas and the dists.
    alpha = 1.- torch.exp(-torch.einsum('brzs, z -> brzs', sigmas, dists))

    # Compute the Ts from the alpha values. Use torch.cumprod.
    Ts = torch.cumprod(1.-alpha + 1e-10, -2)

    # Compute the weights from the Ts and the alphas.
    weights = alpha * Ts

    # Compute the pixel color as the weighted sum of the radiance values.
    rgb = torch.einsum('brzs, brzs -> brs', weights, radiances)

    # Compute the depths as the weighted sum of z_vals.
    # Tip: use torch.einsum for a convenient way of computing the weighted sum,
    # without the need to reshape the z_vals.
    depth_map = torch.einsum('brzs, z -> brs', weights, z_vals)

    return rgb, depth_map, weights


class VolumeRenderer(nn.Module):
    def __init__(self, near, far, n_samples=32, white_back=True):
        super().__init__()
        self.near = near
        self.far = far
        self.n_samples = n_samples
        self.white_back = white_back

    def forward(
        self,
        cam2world,
        intrinsics,
        x_pix,
        radiance_field: nn.Module
        ) -> Tuple[torch.tensor, torch.tensor]:
        """
        Takes as inputs ray origins and directions - samples points along the
        rays and then calculates the volume rendering integral.

        Params:
            input_dict: Dictionary with keys 'cam2world', 'intrinsics', and 'x_pix'
            radiance_field: nn.Module instance of the radiance field we want to render.

        Returns:
            Tuple of rgb, depth_map
            rgb: for each pixel coordinate x_pix, the color of the respective ray.
            depth_map: for each pixel coordinate x_pix, the depth of the respective ray.

        """
        batch_size, num_rays = x_pix.shape[0], x_pix.shape[1]

        # Compute the ray directions in world coordinates.
        # Use the function get_world_rays.
        ros, rds = get_world_rays(x_pix, intrinsics, cam2world)

        # Generate the points along rays and their depth values
        # Use the function sample_points_along_rays.
        pts, z_vals = sample_points_along_rays(self.near, self.far, self.n_samples,
                                                ros, rds, device=x_pix.device)

        # Reshape pts to (batch_size, -1, 3).
        pts = pts.reshape(batch_size, -1, 3)

        # Sample the radiance field with the points along the rays.
        sigma, rad = radiance_field(pts)

        # Reshape sigma and rad back to (batch_size, num_rays, self.n_samples, -1)
        sigma = sigma.view(batch_size, num_rays, self.n_samples, 1)
        rad = rad.view(batch_size, num_rays, self.n_samples, 3)

        # Compute pixel colors, depths, and weights via the volume integral.
        rgb, depth_map, weights = volume_integral(z_vals, sigma, rad)

        if self.white_back:
            accum = weights.sum(dim=-2)
            rgb = rgb + (1. - accum)

        return rgb, depth_map

Last time, we used a different `fit` function to fit our radiance field, because we had separated the renderer and the scene representation.

This time, we will instead write a thin wrapper that combines the `RadianceField` and the `VolumeRenderer` in a single `nn.Module` class so that we can just call it with camera parameters to retrieve an image:

In [ ]:
class RadFieldAndRenderer(nn.Module):
    def __init__(self):
        super().__init__()
        self.rf = LatentRadField(n_features=128)
        self.renderer = VolumeRenderer(near=1., far=2.5, n_samples=100,
                                       white_back=True).cuda()

    def forward(self, model_input):
        xy_pix = model_input['x_pix']
        intrinsics = model_input['intrinsics']
        c2w = model_input['cam2world']

        rgb, depth = self.renderer(
                              c2w,
                              intrinsics,
                              xy_pix,
                              self.rf
                              )
        return rgb, depth

As a sanity check, we will fit only *one* of the SRNs cars with this architecture. We first need a plotting function again that plots the model output and the ground truth image:

In [ ]:
def plot_output_ground_truth(model_output, ground_truth, resolution):
    img, depth = model_output

    img = img[0]
    depth = depth[0]
    gt = ground_truth[0]

    fig, axes = plt.subplots(1, 3, figsize=(18, 6), squeeze=False)
    axes[0, 0].imshow(img.cpu().view(*resolution).detach().numpy())
    axes[0, 0].set_title("Trained MLP")
    axes[0, 1].imshow(gt.cpu().view(*resolution).detach().numpy())
    axes[0, 1].set_title("Ground Truth")

    depth = depth.cpu().view(*resolution[:2]).detach().numpy()
    axes[0, 2].imshow(depth, cmap='Greys')
    axes[0, 2].set_title("Depth")

    for i in range(3):
        axes[0, i].set_axis_off()

    plt.show()

We'll instantiate the dataset with only *one* car:

In [ ]:
img_sl = 64

dataset = SRNsCars(max_num_instances=1, img_sidelength=img_sl)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4)

We'll use a simple mse loss again:

In [ ]:
def mse_loss(mlp_out, gt, model):
    img, depth = mlp_out
    return ((img - gt)**2).mean()

Let's fit!


**Note: Due to the limitations in GPU memory that Google Colab comes with, we have to adjust the following hyper-parameters to save memory: low-resolution voxelgrids, low-resolution images, few samples in the volumetric renderer, and a relatively weak 3D convolutional neural network decoder. If we would run this on a faster & larger GPU, we would get drastically better results!**

Note that if you would like, you can either run this notebook locally (on a machine with a better GPU) or pay $10 for the month to get Colab Pro, which gives you access to much faster GPUs.

In [ ]:
rf_and_renderer = RadFieldAndRenderer().cuda()
optim = torch.optim.Adam(lr=1e-3, params=rf_and_renderer.parameters())

In [ ]:
_ = fit(rf_and_renderer,
        iter(data_loader),
        loss_fn=mse_loss,
        resolution=(img_sl, img_sl, 3),
        plotting_function=plot_output_ground_truth,
        optimizer=optim,
        total_steps=1_001,
        steps_til_summary=100
        )

You should be able to recognize the car after ~1k steps. Note that this reconstrution is likely far from perfect due to our memory and compute constraints. Your output quality would most likely look like this:

<div>
<img src="https://drive.google.com/uc?id=1-eVNtUydv72qpbQYnVXc8APGgsfYJ9x9" width="200"/>
</div>

Note that just like with the `LatentFeatureGrid` from above that we used to overfit on a single image, this only fits a *single* latent voxel grid for a *single* 3D car. But what if, instead, we want to learn a *latent space* of these cars?

In [ ]:
del rf_and_renderer

## Part 2.4: Writing a torch class for turning single-sample models into auto-decoders

Next, we will extend the voxelgrid-conditioned radiance field from the previous assignment to an auto-decoder.

For that, we will have to convert the class `RadFieldAndRenderer` into an auto-decoder. What we would like to do is to turn the `self.grid` parameter of the `HyrbidVoxelNeuralField` into an `nn.Embedding` again, so that we can store a *stack* of voxelgrids. Or, we might even consider having these voxelgrids be the output of a convolutional decoder, such that we compress each voxelgrids into a *single* latent.

Alas, in the current framework, that would be hard to do, as these classes are nested: the `RadianceField` owns the `HyrbidGroundPlanNeuralField` class which in turn manages the `self.grid` parameter. If we would turn that into an embedding, we would have to modify all the `forward` passes to take an additional `index` parameter, and write a bunch of new code, which might introduce new bugs...

It turns out there is an easier way. Pytorch has recently introduced a handy little function called `functional_call` - read through the documentation [here](https://pytorch.org/docs/stable/generated/torch.nn.utils.stateless.functional_call.html). This function allows us to call a module's forward pass, as we normally would, *but also pass in a dictionary that maps the name of a parameter to an alternative value that we would like the module to use*!

Let's take a look. We will instantiate a new `LatentFeatureGrid` as in part 1:

In [ ]:
lfg = LatentFeatureGrid(latent_sidelength=1, latent_ch=128, num_up=5, out_ch=3).cuda()

Let's take a look at its parameters:

In [ ]:
def print_params(module):
    for name, param in module.named_parameters():
        print(f"{name}: {tuple(param.shape)}")

print_params(lfg)

`functional_call` now allows us to assemble a dictionary with *alternative parameters* that this module should use in a forward pass!

For instance, we can assemble a dictionary with an *alternative* latent_grid:

In [ ]:
params = {'latent_grid':torch.rand(1, 128, 1, 1).cuda()}

By simply passing this as the second parameter to `functional_call`, we can force the `LatentGrid` module to use that latent grid instead of its own in its forward pass:

In [ ]:
from torch.nn.utils.stateless import functional_call
_ = functional_call(lfg, params, None)

How does that help? It allows us to write a simple wrapper that turns *any* Pytorch Module into an autodecoder! We wrote this one for you, but please do take a look, because you will need it in a second:

In [ ]:
class AutoDecoderWrapper(nn.Module):
    def __init__(
        self,
        num_latents:int, # Number of latent codes for the auto-decoder.
        submodule:nn.Module, # nn.Module instance we want to turn into an auto-decoder.
        param_name:str # Name of the parameter of the nn.Module that we want to replace with the auto-decoded latents.
    ):
        super().__init__()
        trgt_param = dict(submodule.named_parameters())[param_name]
        self.trgt_param_shape = trgt_param.shape[1:]

        # nn.Embedding is like a "list" of parameters: In the forward pass,
        # you can pass a tensor of indices, and this will return the corresponding
        # latent.
        self.latents = nn.Embedding(num_embeddings=num_latents,
                                    embedding_dim=np.prod(self.trgt_param_shape))
        self.latents.weight.data.normal_(0, 1e-1)

        self.param_name = param_name
        self.submodule = submodule

    def forward(self,
                inputs):
        '''
        inputs: dictionary. Expected to have key "idx" for the latent idcs, as well as other inputs for the sub-module.
        '''

        #######
        # TODO

        # Retrieve the dictionary entry "idx" from the input dictionary
        latent_idcs = #### YOUR CODE HERE ####

        batch_size = latent_idcs.shape[0]

        # Use the "idx" variable to index into the self.latents nn.Embedding module
        # and reshape it as (batch_size, target_param_shape[0], ..., target_param_shape[-1])
        params = #### YOUR CODE HERE ####

        # Build a parameter dictionary where self.param_name key holds param as value.
        param_dict = #### YOUR CODE HERE ####

        # call functional_call using the submodule which is the network, the
        # param_dict, and the inputs as arguments,
        output = #### YOUR CODE HERE ####

        # TODO
        #######

        # return the output of the functional call and params
        return output, params


Now, we can use this wrapper to turn `LatentFeatureGrid` into an auto-decoder *without re-writing any of it*. Let's see how that works:

In [ ]:
lfg = LatentFeatureGrid(latent_sidelength=1, latent_ch=128, num_up=5, out_ch=3)
lfg_ad = AutoDecoderWrapper(10_000,
                            lfg,
                            param_name='latent_grid')

The object `lfg_ad` is now an auto-decoder - that means that we can pass it a tensor of integers, it will retrieve the corresponding latent feature grids, and call the object `lfg`, an instance of `LatentFeatureGrid`, with these feature grids:

In [ ]:
test_idcs = torch.randint(0, 100, size=(100, 1))
_ = lfg_ad({"idx":torch.tensor([0])})

In [ ]:
del lfg_ad
del lfg

## Part 2.5: Turning our latent voxel-grid radiance field into an auto-decoder

Equipped with this handy tool, we can now easily build an auto-decoder for our latent radiance fields.

First, let's instantiate our dataset and determine how many cars we want to train on. Let's say, 1,000:

**Note: For this to work, we have to run a reasonable batch size on the order of ~32. Thus, we will reduce the image resolution to save memory - a resolution of 32 really isn't much, but of course, this works with higher resolutions on larger GPUs :)**

In [ ]:
# Hint: While you are debugging, set the number of scenes to "5".
# While you are debugging, set the batch size to "2".
img_sl = 32
num_scenes = 100 # To make this faster for you, we'll only train on 100 cars. This is of course not really enough to learn a good prior, but it'll be much, much faster to train...
batch_size = 32

def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

dataset = SRNsCars(max_num_instances=num_scenes, img_sidelength=img_sl)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          worker_init_fn=worker_init_fn)

Let's instantiate the model for a *single* scene and print its parameters:

In [ ]:
srn_rf = RadFieldAndRenderer()
print_params(srn_rf)

Let's use our AutoDecoderWrapper to turn this into an auto-decoder! Remember to pass in the `num_scenes` parameter.

Below, fill in the blanks to turn this module into an auto-decoder:

In [ ]:
# TODO
# Define srns_ad as AutoDecoderWrapper with num_latents equal to num_scenes,
# submodule as srn_rf, and param_name as rf.scene_rep.grid.latent_grid

srns_ad = #### YOUR CODE HERE ####

print_params(srns_ad)

Let's instantiate our dataset and do a forward pass as a sanity check!

**Note that the previous and this cell are both executed on CPU - that's because it's often much easier to debug certain problems on CPU than on GPU.**

In [ ]:
with torch.no_grad():
    mi, gt = next(iter(data_loader))

    for key, val in mi.items():
        print(key, val.shape)

    test_out = srns_ad(mi)

No error --> looks good ;)

We now need to adapt our auto-decoder loss from earlier to deal with the tuple output from our network:

In [ ]:
def autodecoder_loss(mlp_out, gt, model):
    (img, depth), latents = mlp_out # Have to unpack outputs of autodecoder wrapper

    img_loss = ((img - gt)**2).mean()
    latent_loss = (latents**2).mean()

    param_loss = 0.
    for name, param in model.named_parameters():
        if 'latent' not in name:
            param_loss += (param**2).mean()
    # return img_loss
    return param_loss*1e-4 + latent_loss*1e-4 + img_loss

We similarly need to adapt our `plot_output_ground_truth` function:

In [ ]:
def plot_output_ground_truth(model_output, ground_truth, resolution):
    (img_batch, depth_batch), latent = model_output

    num_batch_items = 4
    fig, axes = plt.subplots(num_batch_items, 3, figsize=(18, 6*num_batch_items), squeeze=False)

    for i in range(num_batch_items):
        img = img_batch[i]
        depth = depth_batch[i]
        gt = ground_truth[i]

        axes[i, 0].imshow(img.cpu().view(*resolution).detach().numpy())
        axes[i, 0].set_title("Trained MLP")
        axes[i, 1].imshow(gt.cpu().view(*resolution).detach().numpy())
        axes[i, 1].set_title("Ground Truth")

        depth = depth.cpu().view(*resolution[:2]).detach().numpy()
        axes[i, 2].imshow(depth, cmap='Greys')
        axes[i, 2].set_title("Depth")

        for j in range(3):
            axes[i, j].set_axis_off()

    plt.show()

Let's fit!

**Note: This will take quite a while to run, somewhere on the order of 15-20 minutes. You will not be graded based on how far this converges - we will judge whether your functions are correct or not based on unit-tests for each of the functions. This result here is just for you to see that it's working!**

**Hint: If you get weird CUDA bugs, try your model out on CPU first! These are often indexing bugs that CUDA can offer limited insight on, but on CPU, the debugging messages will be much more meaningful**

In [ ]:
srns_ad = srns_ad.cuda() # Copy the model to the GPU
optim = torch.optim.Adam(lr=1e-4, params=srns_ad.parameters())

fit(srns_ad,
    iter(data_loader),
    loss_fn=autodecoder_loss,
    resolution=(img_sl, img_sl, 3),
    plotting_function=plot_output_ground_truth,
    total_steps=10_000,
    optimizer=optim,
    steps_til_summary=100
)

You should observe the effect we discussed earlier: The model will converge to a soapbar-style "average car" after \~500 steps, and only after training for a while longer (\~2000 steps) will it start to form the correct shapes & colors.

## Part 2.6 Reconstructing an SRN car from a single image.

With this latent space in hand, you are now able to reconstruct an SRN car from a **single** image - much in the same way as we reconstructed an MNIST digit from only half the image!

This was first demonstrated in the paper "Scene Representation Networks: Continuous 3D-Structure-Aware Neural Scene Representations" - with this course, you've basically caught up all the way (and much further) on what took Vincent 1.5 years :)

As with the MNIST digit, let's pop a single image observation from our dataset and build a generator that only yields that one image:

In [ ]:
img_sl = 64
num_scenes = 1
batch_size = 1

dataset = SRNsCars(max_num_instances=num_scenes, img_sidelength=img_sl)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)


def test_generator():
    sample = next(iter(data_loader))
    while True:
        yield sample

test_dataset = test_generator()

For the optimizer, we want to *only optimize the latent code - not the other parameters of the model*:

In [ ]:
srn_rf.rf.scene_rep.grid.latent_grid.data.normal_()

optim = torch.optim.Adam(lr=1e-3,
                         params=[srn_rf.rf.scene_rep.grid.latent_grid])

We need to adapt our loss again, as this model does not return a latent:

In [ ]:
def test_rec_loss(mlp_out, gt, model):
    (img, depth) = mlp_out # Have to unpack outputs of autodecoder wrapper

    img_loss = ((img - gt)**2).mean()
    return img_loss

def test_plotting(model_output, ground_truth, resolution):
    (img_batch, depth_batch) = model_output

    num_batch_items = 1
    fig, axes = plt.subplots(num_batch_items, 3, figsize=(18, 6*num_batch_items), squeeze=False)

    i = 0
    img = img_batch[i]
    depth = depth_batch[i]
    gt = ground_truth[i]

    axes[i, 0].imshow(img.cpu().view(*resolution).detach().numpy())
    axes[i, 0].set_title("Trained MLP")
    axes[i, 1].imshow(gt.cpu().view(*resolution).detach().numpy())
    axes[i, 1].set_title("Ground Truth")

    depth = depth.cpu().view(*resolution[:2]).detach().numpy()
    axes[i, 2].imshow(depth, cmap='Greys')
    axes[i, 2].set_title("Depth")

    for j in range(3):
        axes[i, j].set_axis_off()

    plt.show()

We can now reconstruct via fitting:

**NOTE: the quality of this reconstruction is going to be somewhat poor, because we have only trained on a dataset of 100 cars. We would expect this to be *a lot* better if we had trained on more - in fact, this setup is very, very similar to that of "Scene Representation Networks", and with a bit of tweaking should yield similar results. If you have time and a bigger GPU, go ahead and try it!**

In [ ]:
fit(srn_rf,
    iter(test_dataset),
    loss_fn=test_rec_loss,
    resolution=(img_sl, img_sl, 3),
    plotting_function=test_plotting,
    total_steps=5_000,
    optimizer=optim,
    steps_til_summary=100
)

# Rendering the car from multiple view points

In [ ]:
f = h5py.File('cars_train.hdf5', 'r')

instances = sorted(list(f.keys()))
idx = 0

key = instances[idx]
instance = f[key]
c2ws_ds = instance['pose']
c2w_keys = list(c2ws_ds.keys())
intrinsics = parse_intrinsics( instance['intrinsics.txt'] ).to(device)
intrinsics[:2, :3] /= 128.

x_pix = get_opencv_pixel_coordinates(128, 128)
x_pix = x_pix[32:-32, 32:-32]

x_pix = resize(x_pix,
               (64, 64),
               anti_aliasing=False,
               order=0)
x_pix = torch.tensor(x_pix).reshape(1, -1, 2).to(device)

poses = []
for i in range(len(c2w_keys)):
    c2w = parse_pose( c2ws_ds[c2w_keys[i]] )
    poses.append(c2w)


poses = torch.stack(poses).to(device)


with torch.no_grad():
    frames = []
    for i in range(len(poses)):
        model_in = {'cam2world': poses[i:i+1], 'intrinsics': intrinsics[None, ...], 'x_pix': x_pix}
        rgb, depth = srn_rf(model_in)

        rgb = rgb.reshape(64, 64, 3).cpu().numpy()
        frames.append(torch.tensor(rgb))



In [ ]:
# We show a grid with the car viewed from multiple viewpoints even though
# we only observed a single image of a car.
output = torchvision.utils.make_grid(torch.stack(frames).permute(0, -1, 1, 2), 10)

plt.figure(figsize=(20, 20))
plt.imshow(output.permute(1, 2, 0))

# The End
This concludes Assignment 3. In summary, you have learned:


1.   A convolutional decoder
2.   How to use a convolutional decoder to decode a latent into a 2D or 3D grid
3.   How to *manually* convert a single-latent module into an auto-decoder by swapping out an nn.Parameter with an nn.Embedding and modifying the forward pass
4.   How to train a 2D auto-decoder on images.
5.   How to use the trained auto-decoder for prior-based reconstruction from an incomplete observation.
6.   How to use a 3D convolutional decoder to build a hybrid discrete-continuous neural field based on a voxelgrid.
7.   How to convert that into an Auto-Decoder by writing a wrapper function instead of manually editing.
8.   How to train a latent space of **3D Scenes** solely via differentiable rendering.
9.   How to use that latent space for **single-image 3D reconstruction**,thereby reproducing the key result from the paper "Scene Representation Networks: Continuous 3D-Structure-Aware Neural Scene Representations".

# Submission Instructions

1.   Click "Runtime -> Run all" and make sure all cells run without an error (you can tell by whether the *final* cell was executed. You can then be confident that you have implemented everything you were supposed to implement, or at least deleted the "raise NotImplementedError".
2.   Once you're certain, click "File -> Download -> Download .ipynb" and "File -> Download -> Download .py"
3.   Log in to Canvas
4.   Upload the `.py` and `.ipynb` file in a `.zip` archive with the filename being your Kerberos ID (e.g. `prafull.zip`).

In [ ]:
|